# 1. (6 points) Create your class that implements the Gradient Boosting concept, based on the locally weighted regression method (Lowess class), and that allows a user-prescribed number of boosting steps. 

The class you develop should have all the mainstream useful options, including “fit,” “is_fitted”,  and “predict,” methods.  Show applications with real data for regression, 10-fold cross-validations and compare the effect of different scalers, such as the “StandardScaler”, “MinMaxScaler”, and the “QuantileScaler”.  In the case of the “Concrete” data set, determine a choice of hyperparameters that yield lower MSEs for your method when compared to the eXtream Gradient Boosting library.

 

In [250]:
import numpy as np
import pandas as pd
from math import ceil
from scipy import linalg
from sklearn.decomposition import PCA
from scipy.spatial import Delaunay
from scipy.interpolate import interp1d, RegularGridInterpolator, griddata, LinearNDInterpolator, NearestNDInterpolator
from sklearn.preprocessing import StandardScaler, MinMaxScaler, QuantileTransformer
from sklearn.model_selection import train_test_split as tts, KFold, GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error as mse
from sklearn import linear_model
from sklearn.utils.validation import check_is_fitted

#Using kernels, distance, weight, and Lowess code from class

# Gaussian Kernel
def Gaussian(x):
  return np.where(np.abs(x)>4,0,1/(np.sqrt(2*np.pi))*np.exp(-1/2*x**2))
# this is the correct vectorized version
def tricubic(x):
  return np.where(np.abs(x)>1,0,(1-np.abs(x)**3)**3)
# Epanechnikov Kernel
def Epanechnikov(x):
  return np.where(np.abs(x)>1,0,3/4*(1-np.abs(x)**2))
# Quartic Kernel
def Quartic(x):
  return np.where(np.abs(x)>1,0,15/16*(1-np.abs(x)**2)**2)

def weight_function(u,v,kern=Epanechnikov,tau=0.1):
    return kern(dist(u,v)/(2*tau))

def dist(u,v):
  D = []
  if len(v.shape)==1:
    v = v.reshape(1,-1)
  # we would like all the pairwise combinations if u and v are matrices
  # we could avoid two for loops if we consider broadcasting
  for rowj in v:
    D.append(np.sqrt(np.sum((u-rowj)**2,axis=1)))
  return np.array(D).T

def kernel_function(xi,x0,kern, tau):
    return kern((xi - x0)/(2*tau))

class Lowess:
    def __init__(self, kernel = Gaussian, tau=0.05):
        self.kernel = kernel
        self.tau = tau

    def fit(self, x, y):
        kernel = self.kernel
        tau = self.tau
        self.xtrain_ = x
        self.yhat_ = y

    def predict(self, x_new):
        check_is_fitted(self)
        x = self.xtrain_
        y = self.yhat_
        lm = linear_model.Ridge(alpha=0.001)
        w = weight_function(x,x_new,self.kernel,self.tau)

        if np.isscalar(x_new):
          lm.fit(np.diag(w)@(x.reshape(-1,1)),np.diag(w)@(y.reshape(-1,1)))
          yest = lm.predict([[x_new]])[0][0]
        else:
          n = len(x_new)
          yest_test = np.zeros(n)
          #Looping through all x-points
          for i in range(n):
            lm.fit(np.diag(w[:,i])@x,np.diag(w[:,i])@y)
            yest_test[i] = lm.predict([x_new[i]])
        return yest_test

In [275]:
#XGBoost setup for comparison:
import xgboost

#Custom Gradient boost model for Lowess
class GradientBoost:
    
    def __init__(self,model=Lowess,scaler=StandardScaler(),tau=0.5,kernel=Gaussian):  
        self.model = model
        self.scaler = scaler
        self.tau = tau
        self.kernel = kernel
        
    def fit(self,X,y):
        if self.scaler:
            self.X = self.scaler.fit_transform(X)
            self.y = y.reshape(-1,1)
        else:
            self.X = X
            self.y = y.reshape(-1,1)
        
        
    def is_fitted(self):
        try:
            if self.X.any() and self.y.any():
                return True
            else:
                return False
        except:
            raise Exception("Data must be fitted first")
        
    def predict(self, xnew, iterations):
        if self.is_fitted():
            pass
        else: 
            raise Exception("Data must be fitted first")
            
        if self.scaler:
            xnew = self.scaler.fit_transform(xnew)
        
        output = np.zeros((iterations,xnew.shape[0]))
        for i in range(iterations):    
            model1 = self.model(tau=self.tau, kernel=self.kernel)
            model1.fit(self.X,self.y)
            residuals = self.y - (model1.predict(self.X).reshape(-1,1))
            model2 = self.model(tau=self.tau, kernel=self.kernel)
            model2.fit(self.X,residuals)
            output[i] = np.add(model1.predict(xnew),model2.predict(xnew))
        return np.mean(output,axis=0)

gradBoost = GradientBoost()

data = pd.read_csv('../content/01intro/concrete.csv')
y = data['strength'].values
X = data.drop(['strength'],axis=1).values


# Performing 10-fold cross-validation on the training data
def KFoldProcessForTau(X,y,tau,iterations,n_quantiles,kern=Gaussian):
    
    gradBoost = GradientBoost(tau=tau,scaler=QuantileTransformer(n_quantiles=n_quantiles),kernel=kern)
    model_xgboost = xgboost.XGBRFRegressor()
    
    X_train, X_test, y_train, y_test = tts(X, y, test_size=0.3, random_state=42)
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    gradBoostMSES = []
    XGBoostMSES = []
    for train_index, val_index in kf.split(X_train):
        X_train_fold, X_test_fold = X_train[train_index], X_train[val_index]
        y_train_fold, y_test_fold = y_train[train_index], y_train[val_index]
    
        gradBoost.fit(X_train_fold, y_train_fold)
        gradBoostYHat = gradBoost.predict(X_test_fold, iterations=iterations)
        gradBoostMSES.append(mse(y_test_fold,gradBoostYHat.reshape(-1,1)))
    
        model_xgboost.fit(X_train_fold,y_train_fold)
        XGBoostYHat = model_xgboost.predict(X_test_fold)
        XGBoostMSES.append(mse(y_test_fold,XGBoostYHat))

    print("Gradboost MSE is: " + str(np.mean(gradBoostMSES)))
    print("XGBoost MSE is: " + str(np.mean(XGBoostMSES)))

iterations = 2
n_quantiles = 70
kern = Gaussian
#A search of tau from 0.01-1 revealed 0.25 to be fairly optimal
tau = 0.25
print("For tau of " + str(tau) + " and boost iterations of " +str(iterations) + " and n scaling quantiles of " + str(n_quantiles) + " for QuantileTransformer scaling")
KFoldProcessForTau(X,y,tau,iterations,n_quantiles,kern)


For tau of 0.25 and boost iterations of 2 and n scaling quantiles of 70 for QuantileTransformer scaling
Gradboost MSE is: 37.591693083352524
XGBoost MSE is: 39.099963288898365


As we can see, we have beaten XGBoost with our custom GradientBoosting class with the Gaussian kernel and two boosting iterations and tau = 0.25 and QuantileTransformer scaling with 70 quantiles

# 2. (3 points) Based on the Usearch library, create your own class that computes the k_Nearest Neighbors for Regression.

In [389]:
from usearch.index import search, MetricKind, Matches, BatchMatches
from sklearn.neighbors import KNeighborsRegressor

class CustomKNN:
    
    def __init__(self, n_neighbors=10, scaler=StandardScaler()):
        self.k = n_neighbors
        self.scaler = scaler
        
        
    def fit(self,X, y):
        if self.scaler:
            self.X = self.scaler.fit_transform(X)
            self.y = y
        else:
            self.X = X
            self.y = y
        
    def is_fitted(self):
        try:
            if self.X.any():
                return True
            else:
                return False
        except:
            raise Exception("Data must be fitted first")    
    
    def n_closest_points(self):
        n = self.X.shape[0]
        
        points_dict = {}
        
        for i, test_point in enumerate(self.X):
            
            smallest_distances = {}
            
            
            distances_from_point = search(self.X, test_point,n,MetricKind.L2sq, exact=True).distances
            indices = search(self.X, test_point,n,MetricKind.L2sq, exact=True).keys
            for idx, distance in enumerate(distances_from_point[1:self.k]):
                smallest_distances[indices[idx+1]] = distance
            points_dict[i] = smallest_distances
            
        #Returns dictionary with key = point1 and value = another dictionary where keys are the n_neighbors closest
        #points to point1 and the values are their distances from point1 
        return points_dict
    
    def predict(self, xnew):
        if self.scaler:
            xnew = self.scaler.fit_transform(xnew)
        
        n = xnew.shape[0]
        
        yPreds = np.ndarray((n,1))
        for row in xnew:
            distances_from_point = search(self.X, row,self.k,MetricKind.L2sq, exact=True).distances
            indices = search(self.X, row,self.k,MetricKind.L2sq, exact=True).keys
            distanceWeights = (1/(distances_from_point))
            yPred = distanceWeights@y[indices].reshape(-1,1)/np.sum(distanceWeights)
            yPreds[i] = yPred
        return yPreds
            
        
        
data = pd.read_csv('../content/01intro/concrete.csv')
X = data.drop(['strength'],axis=1).values
y = data['strength'].values
custK = CustomKNN(n_neighbors=10,scaler=StandardScaler())
defaultK = KNeighborsRegressor(n_neighbors=10)

X_train, X_test, y_train, y_test = tts(X, y, test_size=0.3, random_state=42)
kf = KFold(n_splits=10, shuffle=True, random_state=42)
knnMses = []
defaultMses = []
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_test_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_test_fold = y_train[train_index], y_train[val_index]
    custK.fit(X_train_fold,y_train_fold)
    defaultK.fit(X_train_fold,y_train_fold)
    yhat1 = custK.predict(X_test_fold)
    #print(np.column_stack([y_test_fold,yhat]))
    yhat2 = defaultK.predict(X_test_fold)
    knnMses.append(mse(y_test_fold, yhat2))
    defaultMses.append(mse(y_test_fold,yhat2))
    
np.mean(knnMses)
#np.mean(defaultMses)

100.67348019170473

In [304]:
data

,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30
...,...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.28
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.18
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.70
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.77
